In [0]:
import dlt
from pyspark.sql.functions import col, current_timestamp


In [0]:
@dlt.view
def business_chi_src():
    
    df = spark.readStream.table("workspace.food_inspection_project.silver_business_chi")

    df = df.where(col("license_number").isNotNull())

    
    df = (
        df.withColumn("business_nk", col("license_number").cast("string"))
          .withColumn("event_ts", current_timestamp())
    )


    return df.select(
        "business_nk",   
        "dba_name",
        "aka_name",
        "event_ts"       
    )


In [0]:

dlt.create_streaming_table("business_chi_type2_stage")

dlt.apply_changes(
    target="business_chi_type2_stage",
    source="business_chi_src",
    keys=["business_nk"],
    sequence_by="event_ts",
    ignore_null_updates=True,
    stored_as_scd_type=2      
)


In [0]:
@dlt.table(name="dim_business_chi_scd2")
def dim_business_chi_scd2():
    df = dlt.read_stream("business_chi_type2_stage")

    return (
        df
        .withColumnRenamed("__START_AT", "effective_start_dt")
        .withColumnRenamed("__END_AT", "effective_end_dt")
        .withColumnRenamed("__IS_CURRENT", "is_current")
    )
